"""SST simulator case I.

    Basic simulator of an organization of Stand By, stabilization 
    and transport of a cryonic case.
    Where the system (SST) is composed of:
    + a subject (patient)
    + three agents (doctor, thanatopractician, ambulance)
    + a variable set of time ranges.
    The objective is to simulate a set of cases to extract the 
    average and statistics to decide how to improve a 
    cryonic suspension procedure.
    
    NOTE: this is ideal case (Die by old age, euthanasia on public hospital, 
    perfect collaboration between center and SST, perfect timing between steps, 
    all costs paid, all documents and arrangements made It.

"""

In [1]:
# Imports
import simpy
import random
import statistics
from colorama import init
from termcolor import colored

In [2]:
# Variables
wait_times = []
population = 50 # Stimation of most probable ative cryonicist in Spain, please modify this parameter per country

"""SST.

    SST = Stand By, Stabilization, Transport
    Embalmer = thanatopractitioner
    Medico = dispensing death certificate and embalming
    Stand By = metabolic support and hypothermia of the patient, 
    manual (voluntary) or mechanical (ECMO, LUCA)
    Stabilisation = controlled perfusion of cryoprotective fluids, 
    following IC protocol and formulas for reasons of economy.

    This class instantiates the environment with its constituent 
    agents to iterate their actions with varying times, simulate good and bad times, 
    collect them and make a statistic of the average time. 

    Allowing to make simulations ahead of any practice and 
    thus reduce incidences or hazards for the patient.

"""

In [3]:
class Sst(object):
    # Environment
    def __init__(self, env, agent_embalmer, agent_doctor, agent_ambulance, agent_place):
        self.env = env
        self.embalmer = simpy.Resource(env, agent_embalmer)
        self.doctor = simpy.Resource(env, agent_doctor)
        self.ambulance = simpy.Resource(env, agent_ambulance)
        self.place = simpy.Resource(env, agent_place)

    def death_authorization(self, patient):
        # Remember: timeout is limited to 60 minutes
        yield self.env.timeout(random.randint(1, 60))

    def embalming_authorization(self, patient):
        yield self.env.timeout(1 / 60)

    def stand_by(self, patient):
        # No more than 1h support
        yield self.env.timeout(random.randint(1, 60))
    
    def perfusion(self, patient):
        # Time is optimizing for only head perfusion.
        yield self.env.timeout(random.randint(1, 60))

"""Deanimation.

    State of legal death of patiente.
    Call the variables and instance of environment, patient and SST.
    Simulate de arrivel, embalmer taking the death authorization, 
    embalming authorization and open option of start Stand By 
    under doctor control. Take the generated times by step.
    
    In resume: start the event of cryonics suspenson by deanimation
    and take the generated data with random factor times to generate
    all possible states of every case.
    
"""

In [4]:
def deanimation(env, patient, sst):
    # patient arrives at the Sst
    arrival_time = env.now

    if random.choice([True, False]): # Hypotetical case of extreme delay and simulate the doctor decision of
        with sst.doctor.request() as request: # apply stand by or straigh freeze.
            yield request
            yield env.process(sst.stand_by(patient))
            
    with sst.embalmer.request() as request:
        yield request
        yield env.process(sst.death_authorization(patient))

    with sst.ambulance.request() as request:
        yield request
        yield env.process(sst.embalming_authorization(patient))
    
    with sst.place.request() as request:
        yield request
        yield env.process(sst.perfusion(patient))

    # patient heads into the Sst
    wait_times.append(env.now - arrival_time)

"""Run SST.

    Function who runs the procedure of execution environment.
    Take the constants and variables Environment, Embalmer,
    Doctor, ambulance.
    
    Launch one deanimation per patient.
    And generate another case after ending the last.
    For take enought data for statistics.

"""

In [5]:
def run_Sst(env, agent_embalmer, agent_doctor, agent_ambulance, agent_place, population):
    # Execute instantiation environment and get agents to simulate.
    sst = Sst(env, agent_embalmer, agent_doctor, agent_ambulance, agent_place)

    for patient in range(population): # Stimated population of active possible cryonicist in a country
        env.process(deanimation(env, patient, sst))
    
    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new patient in simulation.

        patient += 1
        env.process(deanimation(env, patient, sst))

"""Get average wait time.
    
    Function with procedure to generate the average time and 
    feed the result to show general average time of 
    suspension.
    
"""

In [6]:
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

"""Get user input.

    Function with procedure to control input
    of flow chart in environment.
    It is pre-configured to not interaction
    with user.
    
"""

In [7]:
def get_user_input():
    agent_embalmer = "1"
    agent_doctor = "1"
    agent_ambulance = "1"
    agent_place = "1"
    params = [agent_embalmer, agent_doctor, agent_ambulance, agent_place]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. Simulation will use default values:",
            "\n1 embalmer, 1 doctor, 1 ambulance.",
        )
        params = ["1", "1", "1"]
    return params

"""Main.

    Main function launch and run all simulation, process and show the results.
    
    Next objectives:
    
     + Add param times ambulance.
     + Add individual times of doctor, embalmer, ambulance, wait time at funeral home, fly shipping.
     + Add readable and markdown color table he general and specific data.
     
"""

In [8]:
def main():
    # Setup
    random.seed(42)
    agent_embalmer, agent_doctor, agent_ambulance, agent_place = get_user_input()

    # Run the simulation
    env = simpy.Environment()
    env.process(run_Sst(env, agent_embalmer, agent_doctor, agent_ambulance, agent_place, population))
    env.run(until=90)
    # View the results
    mins, secs = get_average_wait_time(wait_times)
    print(colored(f"Based in stimated population in country: {population}","grey","on_blue"))
    print(colored(
        "Running simulation...                \n"+
        "Protocol:                            \n"+
        "1º death authorization               \n"+
        "2º embalming authorization           \n"+
        "3º stand by                          \n"+
        "4º stabilization/perfusion           \n"+
        f"\nThe average wait time is {mins} minutes and {secs} seconds.","grey","on_blue")
    )

In [9]:
if __name__ == "__main__":
    main()

Based in stimated population in country: 50
Running simulation...                
Protocol:                            
1º death authorization               
2º embalming authorization           
3º stand by                          
4º stabilization/perfusion           

The average wait time is 53 minutes and 1 seconds.


In [10]:
# Resources:
# https://github.com/realpython/materials/blob/master/simulation-with-simpy/simulate.py
# https://simpy.readthedocs.io/en/latest/
# https://www.cryonics.org/ci-landing/guide-to-cryonics-procedures/
# https://sociedadcrionica.org
# https://pypi.org/project/colorama/

In [ ]:
# Notes:
# Stimation of cryonicist population in Spain: 100, but between people who can't afford or delay cut by !/2.